In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder
import pickle

In [2]:
## Load the dataset
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
## Preprocessing the data
# Drop the irrevelant columns
data=data.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [4]:
## Change gender label to numerical value and Encode Categorical Value
linearencoder_gender = LabelEncoder()
data['Gender'] = linearencoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [6]:
## OHE -> Geography as the geography is in string to convert it into numerical value
oheencoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_e = oheencoder_geo.fit_transform(data[['Geography']]).toarray()
geo_e_df = pd.DataFrame(geo_e,columns=oheencoder_geo.get_feature_names_out(['Geography']))
geo_e_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
## comnibe one hot encoder columns data in original data
data = pd.concat([data.drop('Geography',axis=1),geo_e_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

scale = StandardScaler()
X_train=scale.fit_transform(X_train)
X_test=scale.transform(X_test)

In [12]:
## Saves the encoder and scalers
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(linearencoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(oheencoder_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scale,file)

#### ANN Regression problem statement

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

In [21]:
model = Sequential([
    Input(shape=(X_train.shape[1],)),  # Add an explicit Input layer
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

In [22]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
## Set up TensorBoard
import datetime

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)


In [26]:
## Set up EarlyStopping

early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [27]:
### Training the model

history = model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 101165.8906 - mae: 101165.8906 - val_loss: 98466.2266 - val_mae: 98466.2266
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 99846.5234 - mae: 99846.5234 - val_loss: 96581.8828 - val_mae: 96581.8828
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 97915.8906 - mae: 97915.8906 - val_loss: 91833.5234 - val_mae: 91833.5234
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 91252.6875 - mae: 91252.6875 - val_loss: 83946.0000 - val_mae: 83946.0000
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 82873.5156 - mae: 82873.5156 - val_loss: 74262.4531 - val_mae: 74262.4531
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 73986.5469 - mae: 73986.5469 - val_loss: 64889.7812 - val_mae: 64889.7812
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 983us/step - loss: 63320.1055 - mae: 63320.1055 - val_loss: 57641.5664 - val_mae: 57641.5664
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 977us/s

In [28]:
## Load extension of tensorboard
%load_ext tensorboard

In [ ]:
%tensorboard --logdir regressionlogs/fit

In [31]:
## Evaluate model on the test data
test_loss, test_mea = model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mea}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 893us/step - loss: 51064.9414 - mae: 51064.9414
Test MAE : 50336.7109375


In [32]:
model.save('regression_model.keras')